In [ ]:
# !pip3 install -U pymongo
# !pip install datetime

In [ ]:
# Connect with client
from pymongo import MongoClient
client = MongoClient('localhost',27017) ## or MongoClient("localhost:27")

#Create a data base call "final5400"
db = client.final5400

In [ ]:
# import libraries
import json
from pprint import pprint
import pandas as pd
from bs4 import BeautifulSoup
import re

# Stock_data

In [ ]:
# stock price data file path 
file = "/Users/xianglinlusam/Desktop/5400 Final/dawjones_tenyears2.json"

In [ ]:
# Read the data as json
with open(file) as j:
    json_data = json.loads(j.read())

# The json data is a nested dictionary, therefore, we needed to extract the values of the data
# Only extract the values from the json data
data = list(json_data.values())
data

In [ ]:
# Initialize a collection in db called stock_data
collection_stock = db.stock_data

In [ ]:
# Insert the json file into the collection
collection_stock.insert_many(data)

In [ ]:
#See the list of collection in database. 
names = db.list_collection_names()
print(names)

In [ ]:
#Amount of documents in each database
print(db.stock_data.count_documents({}))
print(db.news_data.count_documents({}))

In [ ]:
# Try query the data using keys 
query_news = collection_news.find({'ticker': 'AAPL', 'date' : '2022-02-11'})
query_stock = collection_stock.find({'ticker': 'AAPL', 'date' : '2022-02-11'})

#pprint make it look better. 

for x in query_news :
    print(x)
    
for x in query_stock:
    pprint(x)

# News_data

In [ ]:
# Refer to Bengzinga files. 

# read csv as df, force it into pandas dataframes
df = pd.read_csv('/Users/xianglinlusam/Desktop/5400 Final/AAPL_benzinga_2020_to_2023.csv')
clean_data = pd.DataFrame(df)

# print 'body' column 
clean_data['body']

In [ ]:
# clean news body 
for i, row in clean_data.iterrows():
    html_text = row['body']
    parsed_html = BeautifulSoup(html_text, 'html.parser')
    text = parsed_html.get_text()
    clean_text = re.sub(r'\s+', ' ', text)
    clean_data.at[i, 'body'] = clean_text 
    
print(clean_data['body'])

In [ ]:
# output the column names 
print(clean_data.columns)

In [ ]:
# print the entire dataframe
print(clean_data)

In [ ]:
# select columns as features
new_df = df[['author','updated','title','body','url']]
#assign 'AAPL' to new columns, since these are all Apple news data
new_df['ticker'] = 'AAPL'
# Change the updated column name to date
new_df.rename(columns = {'updated':'date'}, inplace = True)
print(new_df)

In [ ]:
# Change the format of date to year-month-day
new_df['date'] = pd.to_datetime(new_df['date']).dt.strftime('%Y-%m-%d')

In [ ]:
print(new_df.head(10))

In [ ]:
new_df.to_csv('clean_AAPL.csv', index=False)

In [ ]:
# Change to dictionary format
new_df_dict = new_df.to_dict('records')

In [ ]:
# write it into json
news_json = json.dumps(new_df_dict, indent = 4)

# save the file local
with open('data.json', 'w') as file:
    file.write(news_json)

In [ ]:
# path for the saved json file
news = '/Users/xianglinlusam/Desktop/5400 Final/data.json'

In [ ]:
# open the json file and print it out
with open(news) as j_news:
    json_news = json.loads(j_news.read())
    
json_news

In [ ]:
# add the news data from json into database. 
collection_news = db.news_data
collection_news.insert_many(json_news)

# News_NLP

In [ ]:
# Refer to the spark file

# path
data_nlp_path = "/Users/xianglinlusam/Desktop/5400 Final/NLP Data with predicted Stock Price"

In [ ]:
# read data to csv
new_data = pd.read_csv(data_nlp_path)

# dataframe to dictionary
new_data_dict = new_data.to_dict('records')


# write into json format then save it locally
news_NLP_json = json.dumps(new_data_dict)

with open('NLPdata.json', 'w') as file:
    file.write(news_NLP_json)

In [ ]:
# local path 
news_NLP = '/Users/xianglinlusam/Desktop/5400 Final/NLPdata.json'

In [ ]:
# load the file, and print it out 
with open(news_NLP) as j_news:
    json_news_nlp = json.loads(j_news.read())
    
json_news_nlp

In [ ]:
# add the news data into json database
collection_news_NLP = db.news_NLP
collection_news_NLP.insert_many(json_news_nlp)